# Statistical Hypothesis Testing

In [7]:
import math
from lib.probability import (
    normal_cdf,
    inverse_normal_cdf
)


# Example: Flipping a Coin
# Imagine we have a coin and we want to test whether it’s fair. We’ll make the assump‐
# tion that the coin has some probability p of landing heads, and so our null hypothesis 
# is that the coin is fair—that is, that p = 0 . 5. We’ll test this against the alternative
# hypothesis p ≠ 0 . 5

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# the normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) 
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2
    # upper bound should have tail_probability  above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

In [8]:
# In particular, let’s say that we choose to flip the coin n = 1000 times. If our hypothesis
# of fairness is true, X should be distributed approximately normally with mean 50 and
# standard deviation 15.8:

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print('mu : ', mu_0)
print('sigma : ', sigma_0)

mu :  500.0
sigma :  15.811388300841896


In [24]:
# We need to make a decision about significance—how willing we are to make a type 1
# error (“false positive”), in which we reject H 0 even though it’s true. For reasons lost to
# the annals of history, this willingness is often set at 5% or 1%. Let’s choose 5%.
# Consider the test that rejects H 0 if X falls outside the bounds given by:

lo, hi = normal_two_sided_bounds(0.5, mu_0, sigma_0)
print("lower bound : ", lo)
print("upper bound : ", hi)

lower bound :  489.3354374162956
upper bound :  510.6645625837044


In [14]:
# Assuming p really equals 0.5 (i.e., H 0 is true), there is just a 5% chance we observe an
# X that lies outside this interval, which is the exact significance we wanted. Said differ‐
# ently, if H 0 is true, then, approximately 19 times out of 20, this test will give the cor‐
# rect result.
# We are also often interested in the power of a test, which is the probability of not
# making a type 2 error, in which we fail to reject H 0 even though it’s false. In order to
# measure this, we have to specify what exactly H 0 being false means. (Knowing merely
# that p is not 0.5 doesn’t give you a ton of information about the distribution of X.) In
# particular, let’s check what happens if p is really 0.55, so that the coin is slightly biased
# toward heads.
# In that case, we can calculate the power of the test with:

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print("mu_1 : ", mu_1)
print("sigma_1 : ", sigma_1)

mu_1 :  550.0
sigma_1 :  15.732132722552274


In [26]:
# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("type_2_probability : ", type_2_probability)
print("power : ", power)

type_2_probability :  0.06356291276992415
power :  0.9364370872300758


In [27]:
# Imagine instead that our null hypothesis was that the coin is not biased toward heads,
# or that p ≤ 0 . 5. In that case we want a one-sided test that rejects the null hypothesis
# when X is much larger than 50 but not when X is smaller than 50. So a 5% significance test 
# involves using normal_probability_below to find the cutoff below
# which 95% of the probability lies:

hi = normal_upper_bound(0.95, mu_0, sigma_0)
# is 526 (< 531, since we need more probability in the upper tail)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("hi : ", hi)
print("type_2_probability : ", type_2_probability)
print("power : ", power)

hi :  526.0073585242053
type_2_probability :  0.06362051966928273
power :  0.9363794803307173


In [28]:
# This is a more powerful test, since it no longer rejects H 0 when X is below 469 (which
# is very unlikely to happen if H 1 is true) and instead rejects H 0 when X is between 526
# and 531 (which is somewhat likely to happen if H 1 is true). === p-values
# An alternative way of thinking about the preceding test involves p-values. Instead of
# choosing bounds based on some probability cutoff, we compute the probability—
# assuming H 0 is true—that we would see a value at least as extreme as the one we
# actually observed.
# For our two-sided test of whether the coin is fair, we compute:

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

In [31]:
import random


# One way to convince yourself that this is a sensible estimate is with a simulation:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
print("{} / 100000 = {}".format(extreme_value_count, extreme_value_count/100000))

6274 / 100000 = 0.06274
